# Обработка инцидентов

Есть набор из N инцидентов. Каждый имеет: id с последовательными значениями от 0 до N-1, два категориальных признака с целыми значениями от 0 до M-1, признак времени с каким-то (нецелым) значением от 0 до 1.

Надо написать на функцию, которая получает на вход dT и файл с инцидентами, а вычисляет для каждого из N инцидентов количество инцидентов, которые удовлетворяют следующим условиям:

- предшествуют данному инциденту по времени, при этом разница по времени не превосходит dT;
- имеют значения feature1 и feature2, совпадающие с соответствующими значениями данного инцидента.

Пример:

id,feature1,feature2,time

0,1,0,0.206520219143

1,0,0,0.233725001118

2,0,1,0.760992754734

3,1,1,0.92776979943

4,1,0,0.569711498585

5,0,1,0.99224586863

6,0,0,0.593264390713

7,1,0,0.694181201747

8,1,1,0.823812651856

9,0,1,0.906011017725


В случае dT=0.3 для приведенного выше примера ответ должен выглядеть так:

id,count

0,0

1,0

2,0

3,1

4,0

5,2

6,0

7,1

8,0

9,1

Функция должна считывать csv-файл с инцидентами, вычислять результаты для всех инцидентов и выписывать их в csv-файл указанного вида. Основной нюанс: функция должна работать достаточно быстро, а именно не дольше минуты при M=100, N=1000000, dT=0.3. Объем потребляемой программой памяти в процессе работы не должен превышать 2Гб.

## Создание файла с инцидентами

In [1]:
# Файл с инцидентом из примера

import numpy as np
import pandas as pd

M = 2
N = 10

df = pd.DataFrame({'feature1':[1,0,0,1,1,0,0,1,1,0],
                   'feature2':[0,0,1,1,0,1,0,0,1,1],
                   'time':[0.206520219143,0.233725001118,0.760992754734,0.92776979943,0.569711498585,0.99224586863,0.593264390713,0.694181201747,0.823812651856,0.906011017725]
                  })

df.to_csv('incidents.csv', index_label='id')

In [2]:
# Файл с инцидентом для проверки времени работы и памяти

import numpy as np
import pandas as pd

M = 100
N = 1000000

df = pd.DataFrame({'feature1':np.random.randint(M, size=(N,)),
                 'feature2':np.random.randint(M, size=(N,)),
                 'time':np.random.rand(N)
                 })

df.to_csv('incidents.csv', index_label='id')

## Решение задачи

In [3]:
%%time

import numpy as np
import pandas as pd
import os
import psutil

process = psutil.Process(os.getpid())

def incidents(incidents_in, incidents_out, dT):
    df = pd.read_csv(incidents_in, index_col='id').values

# Количество строк в df
    n = len(df)

# Индексы для сортировки по time по убыванию
    ix = np.argsort(df[:,2])[::-1]

# Сортируем таблицу по time по убыванию
    df = df[ix]

# Заготовка для итогового файла
    cT = np.zeros(n, dtype=np.int16)

# Индексы уникальных пар (feature1, feature2)
    _, indices, _ = np.unique(df[:,0:2], axis=0, return_counts=True, return_inverse=True)

# Список массивов с индексами уникальных пар (feature1, feature2)
    cff = [np.argwhere(i==indices) for i in np.unique(indices)]

# Список массивов c индексами более 1 уникальной пары (feature1, feature2)
    cfs = [cf.T[0] for cf in cff if len(cf)>1]

# Выбираем те строки, где расстояние между time меньше заданного
    for cf in cfs:
        for i, row in enumerate(cf[:-1]):
# Результат пишем, используя индекс сортировки
            cT[ix[row]] = len([j for j in xrange(1+i,len(cf)) if (df[cf[i],2]-df[cf[j],2])<dT])
    
    pd.DataFrame(cT, columns = ['count']).to_csv(incidents_out, index_label='id')

incidents('incidents.csv', 'cT.csv', 0.3)

# Объем потребляемой программой памяти в байтах
print process.memory_info()[0]

150573056
CPU times: user 1min 27s, sys: 1.83 s, total: 1min 29s
Wall time: 1min 31s


## Проверка

In [5]:
!cat cT.csv

id,count
0,0
1,0
2,0
3,1
4,0
5,2
6,0
7,1
8,0
9,1
